In [1]:
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.appName("Soccer Statistic").getOrCreate()

23/11/05 19:34:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df_matches = spark.read.csv("data/matches.csv", inferSchema=True, header=True)
df_matches.show()

+--------+---+------+----------+------------------+--------------+----+----+---+
|Match_ID|Div|Season|      Date|          HomeTeam|      AwayTeam|FTHG|FTAG|FTR|
+--------+---+------+----------+------------------+--------------+----+----+---+
|       1| D2|  2009|2010-04-04|        Oberhausen|Kaiserslautern|   2|   1|  H|
|       2| D2|  2009|2009-11-01|       Munich 1860|Kaiserslautern|   0|   1|  A|
|       3| D2|  2009|2009-10-04|     Frankfurt FSV|Kaiserslautern|   1|   1|  D|
|       4| D2|  2009|2010-02-21|     Frankfurt FSV|     Karlsruhe|   2|   1|  H|
|       5| D2|  2009|2009-12-06|             Ahlen|     Karlsruhe|   1|   3|  A|
|       6| D2|  2009|2010-04-03|      Union Berlin|     Karlsruhe|   1|   1|  D|
|       7| D2|  2009|2009-08-14|         Paderborn|     Karlsruhe|   2|   0|  H|
|       8| D2|  2009|2010-03-08|         Bielefeld|     Karlsruhe|   0|   1|  A|
|       9| D2|  2009|2009-09-26|    Kaiserslautern|     Karlsruhe|   2|   0|  H|
|      10| D2|  2009|2009-11

In [4]:
# Rename column:
# FTHG --> HomeTeamGoals
# FTAG --> AwayTeamGoals
# FTR  --> FinalResult

old_cols = ["FTHG", "FTAG", "FTR"]
new_cols = ["HomeTeamGoals", "AwayTeamGoals", "FinalResult"]
old_new_cols = [*zip(old_cols, new_cols)]
for old_cols, new_cols in old_new_cols:
    df_matches = df_matches.withColumnRenamed(old_cols, new_cols)

df_matches.show()

+--------+---+------+----------+------------------+--------------+-------------+-------------+-----------+
|Match_ID|Div|Season|      Date|          HomeTeam|      AwayTeam|HomeTeamGoals|AwayTeamGoals|FinalResult|
+--------+---+------+----------+------------------+--------------+-------------+-------------+-----------+
|       1| D2|  2009|2010-04-04|        Oberhausen|Kaiserslautern|            2|            1|          H|
|       2| D2|  2009|2009-11-01|       Munich 1860|Kaiserslautern|            0|            1|          A|
|       3| D2|  2009|2009-10-04|     Frankfurt FSV|Kaiserslautern|            1|            1|          D|
|       4| D2|  2009|2010-02-21|     Frankfurt FSV|     Karlsruhe|            2|            1|          H|
|       5| D2|  2009|2009-12-06|             Ahlen|     Karlsruhe|            1|            3|          A|
|       6| D2|  2009|2010-04-03|      Union Berlin|     Karlsruhe|            1|            1|          D|
|       7| D2|  2009|2009-08-14|     

<h4>Question: Who are the winners of the D1 division in the Germany 
    Football Association (Bundesliga) between 2000–2010</h4>

In [5]:
# Mapping match result to depend on 'FinalResult' to 3 new columns: HomeTeamWin, AwayTeamWin, GameTie
df_matches = df_matches \
.withColumn("HomeTeamWin", when(col("FinalResult") == "H", 1).otherwise(0)) \
.withColumn("AwayTeamWin", when(col("FinalResult") == "A", 1).otherwise(0)) \
.withColumn("GameTie", when(col("FinalResult") == "D", 1).otherwise(0))

In [6]:
# Filter D1 division and season from 2000 - 2010
bundesliga = df_matches.filter(
    (col("Div") == "D1") & 
    (col("Season") >= 2000) & 
    (col("Season") <= 2010)
)

In [83]:
bundesliga.show()

+--------+---+------+----------+-------------+----------+-------------+-------------+-----------+-----------+-----------+-------+
|Match_ID|Div|Season|      Date|     HomeTeam|  AwayTeam|HomeTeamGoals|AwayTeamGoals|FinalResult|HomeTeamWin|AwayTeamWin|GameTie|
+--------+---+------+----------+-------------+----------+-------------+-------------+-----------+-----------+-----------+-------+
|      21| D1|  2009|2010-02-06|       Bochum|Leverkusen|            1|            1|          D|          0|          0|      1|
|      22| D1|  2009|2009-11-22|Bayern Munich|Leverkusen|            1|            1|          D|          0|          0|      1|
|      23| D1|  2009|2010-05-08|   M'gladbach|Leverkusen|            1|            1|          D|          0|          0|      1|
|      24| D1|  2009|2009-08-08|        Mainz|Leverkusen|            2|            2|          D|          0|          0|      1|
|      25| D1|  2009|2009-10-17|      Hamburg|Leverkusen|            0|            0|     

In [7]:
# home team features
home = bundesliga.groupBy("Season", "HomeTeam") \
.agg(
    sum("HomeTeamWin").alias("TotalHomeWin"),
    sum("AwayTeamWin").alias("TotalHomeLoss"),
    sum("GameTie").alias("TotalHomeTie"),
    sum("HomeTeamGoals").alias("HomeScoreGoals"),
    sum("AwayTeamGoals").alias("HomeAgainstGoals")
).withColumnRenamed("HomeTeam", "Team")

# away team features
away = bundesliga.groupBy("Season", "AwayTeam") \
.agg(
    sum("AwayTeamWin").alias("TotalAwayWin"),
    sum("HomeTeamWin").alias("TotalAwayLoss"),
    sum("GameTie").alias("TotalAwayTie"),
    sum("AwayTeamGoals").alias("AwayScoreGoals"),
    sum("HomeTeamGoals").alias("AwayAgainstGoals")
).withColumnRenamed("AwayTeam", "Team")

In [8]:
window = Window.partitionBy(["Season"]).orderBy(col("Points").desc(), col("GoalDifferentials").desc())
teams_result = home.join(away, ["Team", "Season"], "inner") \
.withColumn("Win", col("TotalHomeWin") + col("TotalAwayWin")) \
.withColumn("Tie", col("TotalHomeTie") + col("TotalAwayTie")) \
.withColumn("Loss", col("TotalHomeLoss") + col("TotalAwayLoss")) \
.withColumn("Points", col("Win") * 3 + col("Tie")) \
.withColumn("WinPct", round(col("Win") / (col("Win") + col("Tie") + col("Loss")) * 100, 2)) \
.withColumn("GoalsScored", col("HomeScoreGoals") + col("AwayScoreGoals")) \
.withColumn("GoalsAgainst", col("HomeAgainstGoals") + col("AwayAgainstGoals")) \
.withColumn("GoalDifferentials", col("GoalsScored") - col("GoalsAgainst")) \
.drop("TotalHomeWin", "TotalHomeLoss", "TotalHomeTie", "HomeScoreGoals", "HomeAgainstGoals") \
.drop("TotalAwayWin", "TotalAwayLoss", "TotalAwayTie", "AwayScoreGoals", "AwayAgainstGoals") \
.withColumn("TeamPosition", rank().over(window))

In [ ]:
sql = """
    with league as (
    	select
    		*
    	from "match"
    	where division = 'D1' and season >= '2000' and season <= '2010'
    ), home_result as (
    	select
    		home_team as team,
    		season,
    		sum(home_team_goals) as total_team_goals,
    		sum(away_team_goals) as total_against_goals,
    		sum(CASE WHEN final_result = 'H' THEN 1 ELSE 0 END) AS team_win,
    		SUM(CASE WHEN final_result = 'A' THEN 1 ELSE 0 END) AS team_loss,
    		SUM(CASE WHEN final_result = 'D' THEN 1 ELSE 0 END) AS game_tie
    	from league b1
    	group by home_team, season
    ), away_result as (
    	select
    		away_team as team,
    		season,
    		sum(home_team_goals) as total_against_goals,
    		sum(away_team_goals) as total_team_goals,
    		sum(CASE WHEN final_result = 'H' THEN 1 ELSE 0 END) AS team_loss,
    		SUM(CASE WHEN final_result = 'A' THEN 1 ELSE 0 END) AS team_win,
    		SUM(CASE WHEN final_result = 'D' THEN 1 ELSE 0 END) AS game_tie
    	from league b1
    	group by away_team, season
    )
    select
    	hr.team,
    	hr.season,
    	hr.team_win + ar.team_win as total_win,
    	hr.team_loss + ar.team_loss as total_loss,
    	hr.game_tie + ar.game_tie as total_draw,
    	hr.total_team_goals + ar.total_team_goals as total_goals,
    	hr.total_against_goals + ar.total_against_goals as total_loss_goals,
    	hr.total_team_goals + ar.total_team_goals - (hr.total_against_goals + ar.total_against_goals) as goals_difference,
    	(hr.team_win + ar.team_win) * 3 + (hr.game_tie + ar.game_tie) as points,
    	rank() over (
    		partition by hr.season
    		order by (hr.team_win + ar.team_win) * 3 + (hr.game_tie + ar.game_tie) desc,
    		hr.total_team_goals + ar.total_team_goals - (hr.total_against_goals + ar.total_against_goals) asc
    	) as position
    from home_result hr
    inner join away_result ar on hr.team = ar.team and hr.season = ar.season
    order by season asc
"""

In [88]:
result = teams_result.where(col("TeamPosition") == 1).orderBy(col("Season").asc())
result.show()

+-------------+------+---+---+----+------+------+-----------+------------+-----------------+------------+
|         Team|Season|Win|Tie|Loss|Points|WinPct|GoalsScored|GoalsAgainst|GoalDifferentials|TeamPosition|
+-------------+------+---+---+----+------+------+-----------+------------+-----------------+------------+
|Bayern Munich|  2000| 19|  6|   9|    63| 55.88|         62|          37|               25|           1|
|     Dortmund|  2001| 21|  7|   6|    70| 61.76|         62|          33|               29|           1|
|Bayern Munich|  2002| 23|  6|   5|    75| 67.65|         70|          25|               45|           1|
|Werder Bremen|  2003| 22|  8|   4|    74| 64.71|         79|          38|               41|           1|
|Bayern Munich|  2004| 24|  5|   5|    77| 70.59|         75|          33|               42|           1|
|Bayern Munich|  2005| 22|  9|   3|    75| 64.71|         67|          32|               35|           1|
|    Stuttgart|  2006| 21|  7|   6|    70| 61.

<h4>Question: Which teams have been relegated in the past 10 years</h4>

In [39]:
relegated_teams = teams_result.select("Team").where(col("TeamPosition") >= 16).distinct().withColumnRenamed("Team", "RelegatedTeam")
relegated_teams.show()

+--------------+
| RelegatedTeam|
+--------------+
|         Mainz|
|Kaiserslautern|
|      Nurnberg|
|        Bochum|
|        Hertha|
|  Unterhaching|
|     Bielefeld|
|       Cottbus|
|       FC Koln|
|      Freiburg|
| Ein Frankfurt|
|      Duisburg|
|      St Pauli|
|   Munich 1860|
|        Aachen|
|     Karlsruhe|
|    M'gladbach|
| Hansa Rostock|
+--------------+

